## Rithish Kumar

###Credits to Wav2Lip pretrained model

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Getting Pretrained Wav2Lip program from git.
which can be used for academic and research purposes.

In [3]:
!git clone https://github.com/Rudrabha/Wav2Lip.git

Cloning into 'Wav2Lip'...
remote: Enumerating objects: 338, done.
remote: Total 338 (delta 0), reused 0 (delta 0), pack-reused 338
Receiving objects: 100% (338/338), 511.49 KiB | 12.18 MiB/s, done.
Resolving deltas: 100% (184/184), done.


In [4]:
!ls

gdrive	sample_data  Wav2Lip


In [5]:
!cp -ri "/content/gdrive/MyDrive/Wav2lip/wav2lip_gan.pth" /content/Wav2Lip/checkpoints/

In [15]:
!cp -ri "/content/gdrive/MyDrive/Wav2lip/haarcascade_frontalface_alt2.xml" /content/

# Get the pre-requisites

In [6]:
!pip uninstall tensorflow tensorflow-gpu

Uninstalling tensorflow-2.4.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.4.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.4.1


In [7]:
!cd Wav2Lip && pip install -r requirements.txt

     |████████████████████████████████| 1.6MB 7.2MB/s 
     |████████████████████████████████| 20.3MB 1.2MB/s 
     |████████████████████████████████| 57.4MB 49kB/s 
     |████████████████████████████████| 26.6MB 116kB/s 
     |████████████████████████████████| 676.9MB 26kB/s 
     |████████████████████████████████| 2.6MB 22.3MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 3.6MB 28.7MB/s 
     |████████████████████████████████| 20.2MB 10.3MB/s 
  Created wheel for librosa: filename=librosa-0.7.0-cp37-none-any.whl size=1598345 sha256=938e2a437fa2fbb12128a95b7ec5284b042dd195131a36cacb4013d438286e8b
  Stored in directory: /root/.cache/pip/wheels/49/1d/38/c8ad12fcad67569d8e730c3275be5e581bd589558484a0f881
Successfully built librosa
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.1.0 which is incompatible.
ERROR: datascience 0.10.6 has requir

In [8]:
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "Wav2Lip/face_detection/detection/sfd/s3fd.pth"

--2021-05-24 10:55:39--  https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth
Resolving www.adrianbulat.com (www.adrianbulat.com)... 45.136.29.207
Connecting to www.adrianbulat.com (www.adrianbulat.com)|45.136.29.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89843225 (86M) [application/octet-stream]
Saving to: ‘Wav2Lip/face_detection/detection/sfd/s3fd.pth’

Wav2Lip/face_detect 100%[===================>]  85.68M  18.0MB/s    in 5.6s    

2021-05-24 10:55:45 (15.4 MB/s) - ‘Wav2Lip/face_detection/detection/sfd/s3fd.pth’ saved [89843225/89843225]



In [9]:
%cd /content

/content


#### Function to upload our file to colab.

In [10]:
def upload(savename):
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(savename, 'wb') as f:
      f.write(data)
      print ('saved file', name)
  return name


# Now Lets do some deep fakes

First we are getting ip from user and get to our local storage.


In [11]:
upload('input_raw.mp4')

Saving videoplayback (3).mp4 to videoplayback (3).mp4
saved file videoplayback (3).mp4


'videoplayback (3).mp4'

In [12]:
upload('input_audio.mp4')

Saving videoplayback (4).mp4 to videoplayback (4).mp4
saved file videoplayback (4).mp4


'videoplayback (4).mp4'

## From the main footage we are getting frames with faces.

In [20]:
import cv2
face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_alt2.xml')
  

# Create an object to read
# from camera
video = cv2.VideoCapture('/content/input_raw.mp4')

# We need to check if camera
# is opened previously or not
if (video.isOpened() == False):
	print("Error reading video file")

# We need to set resolutions.
# so, convert them from float to integer.
frame_width = int(video.get(3))
frame_height = int(video.get(4))

size = (frame_width, frame_height)

# Below VideoWriter object will create
# a frame of above defined The output
# is stored in 'filename.avi' file.
result = cv2.VideoWriter('input.mp4',
						cv2.VideoWriter_fourcc(*'MJPG'),
						10, size)
	
while(True):
  ret, frame = video.read()

  if ret == True:
    p=0
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Detects faces of different sizes in the input image
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    c=0
    for face in faces:
      c+=1
# Write the frame into the
# file 'filename.avi'
    if c!=0:
      result.write(frame)


  else:
    break

# When everything done, release
# the video capture and video
# write objects
video.release()
result.release()
	


print("The video was successfully saved")


The video was successfully saved


In [13]:
%cd /content/Wav2Lip/

/content/Wav2Lip


### Setting model and doing lip_sync

In [22]:
from os import listdir, path
import numpy as np
import scipy, cv2, os, sys, argparse, audio
import json, subprocess, random, string
from tqdm import tqdm
from glob import glob
import torch, face_detection
from models import Wav2Lip
import platform
checkpoint_path="checkpoints/wav2lip_gan.pth"

face='/content/input.mp4'
audiof='/content/input_audio.mp4'
outfile='results/result_voice.mp4'

static=False
fps=25
pads=[0,10,0,0]
face_det_batch_size =16
wav2lip_batch_size=128
resize_factor=1
crop=[0, -1, 0, -1]
box=[-1, -1, -1, -1]
rotate=False
nosmooth=False

img_size = 96

if os.path.isfile(face) and face.split('.')[1] in ['jpg', 'png', 'jpeg']:
	static = True

def get_smoothened_boxes(boxes, T):
	for i in range(len(boxes)):
		if i + T > len(boxes):
			window = boxes[len(boxes) - T:]
		else:
			window = boxes[i : i + T]
		boxes[i] = np.mean(window, axis=0)
	return boxes

def face_detect(images):
	detector = face_detection.FaceAlignment(face_detection.LandmarksType._2D, 
											flip_input=False, device=device)

	batch_size =face_det_batch_size
	
	while 1:
		predictions = []
		try:
			for i in tqdm(range(0, len(images), batch_size)):
				predictions.extend(detector.get_detections_for_batch(np.array(images[i:i + batch_size])))
		except RuntimeError:
			if batch_size == 1: 
				raise RuntimeError('Image too big to run face detection on GPU. Please use the --resize_factor argument')
			batch_size //= 2
			print('Recovering from OOM error; New batch size: {}'.format(batch_size))
			continue
		break

	results = []
	pady1, pady2, padx1, padx2 = pads
	for rect, image in zip(predictions, images):
		if rect is None:
			cv2.imwrite('temp/faulty_frame.jpg', image) # check this frame where the face was not detected.
			raise ValueError('Face not detected! Ensure the video contains a face in all the frames.')

		y1 = max(0, rect[1] - pady1)
		y2 = min(image.shape[0], rect[3] + pady2)
		x1 = max(0, rect[0] - padx1)
		x2 = min(image.shape[1], rect[2] + padx2)
		
		results.append([x1, y1, x2, y2])

	boxes = np.array(results)
	if not nosmooth: boxes = get_smoothened_boxes(boxes, T=5)
	results = [[image[y1: y2, x1:x2], (y1, y2, x1, x2)] for image, (x1, y1, x2, y2) in zip(images, boxes)]

	del detector
	return results 

def datagen(frames, mels):
	img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []

	if box[0] == -1:
		if not static:
			face_det_results = face_detect(frames) # BGR2RGB for CNN face detection
		else:
			face_det_results = face_detect([frames[0]])
	else:
		print('Using the specified bounding box instead of face detection...')
		y1, y2, x1, x2 = box
		face_det_results = [[f[y1: y2, x1:x2], (y1, y2, x1, x2)] for f in frames]

	for i, m in enumerate(mels):
		idx = 0 if static else i%len(frames)
		frame_to_save = frames[idx].copy()
		face, coords = face_det_results[idx].copy()

		face = cv2.resize(face, (img_size,img_size))
			
		img_batch.append(face)
		mel_batch.append(m)
		frame_batch.append(frame_to_save)
		coords_batch.append(coords)

		if len(img_batch) >= wav2lip_batch_size:
			img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)

			img_masked = img_batch.copy()
			img_masked[:,img_size//2:] = 0

			img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
			mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])

			yield img_batch, mel_batch, frame_batch, coords_batch
			img_batch, mel_batch, frame_batch, coords_batch = [], [], [], []

	if len(img_batch) > 0:
		img_batch, mel_batch = np.asarray(img_batch), np.asarray(mel_batch)

		img_masked = img_batch.copy()
		img_masked[:, img_size//2:] = 0

		img_batch = np.concatenate((img_masked, img_batch), axis=3) / 255.
		mel_batch = np.reshape(mel_batch, [len(mel_batch), mel_batch.shape[1], mel_batch.shape[2], 1])

		yield img_batch, mel_batch, frame_batch, coords_batch

mel_step_size = 16
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} for inference.'.format(device))

def _load(checkpoint_path):
	if device == 'cuda':
		checkpoint = torch.load(checkpoint_path)
	else:
		checkpoint = torch.load(checkpoint_path,
								map_location=lambda storage, loc: storage)
	return checkpoint

def load_model(path):
	model = Wav2Lip()
	print("Load checkpoint from: {}".format(path))
	checkpoint = _load(path)
	s = checkpoint["state_dict"]
	new_s = {}
	for k, v in s.items():
		new_s[k.replace('module.', '')] = v
	model.load_state_dict(new_s)

	model = model.to(device)
	return model.eval()
# MAIN FUNCTION
if not os.path.isfile(face):
  raise ValueError('--face argument must be a valid path to video/image file')

elif face.split('.')[1] in ['jpg', 'png', 'jpeg']:
  full_frames = [cv2.imread(face)]
  fps = fps

else:
  video_stream = cv2.VideoCapture(face)
  fps = video_stream.get(cv2.CAP_PROP_FPS)

  print('Reading video frames...')

  full_frames = []
  while 1:
    still_reading, frame = video_stream.read()
    if not still_reading:
      video_stream.release()
      break
    if resize_factor > 1:
      frame = cv2.resize(frame, (frame.shape[1]//resize_factor, frame.shape[0]//resize_factor))

    if rotate:
      frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_CLOCKWISE)

    y1, y2, x1, x2 =crop
    if x2 == -1: x2 = frame.shape[1]
    if y2 == -1: y2 = frame.shape[0]

    frame = frame[y1:y2, x1:x2]

    full_frames.append(frame)

print ("Number of frames available for inference: "+str(len(full_frames)))

if not audiof.endswith('.wav'):
  print('Extracting raw audio...')
  command = 'ffmpeg -y -i {} -strict -2 {}'.format(audiof, 'temp/temp.wav')

  subprocess.call(command, shell=True)
  audiof = 'temp/temp.wav'

wav = audio.load_wav(audiof, 16000)
mel = audio.melspectrogram(wav)
print(mel.shape)

if np.isnan(mel.reshape(-1)).sum() > 0:
  raise ValueError('Mel contains nan! Using a TTS voice? Add a small epsilon noise to the wav file and try again')

mel_chunks = []
mel_idx_multiplier = 80./fps 
i = 0
while 1:
  start_idx = int(i * mel_idx_multiplier)
  if start_idx + mel_step_size > len(mel[0]):
    mel_chunks.append(mel[:, len(mel[0]) - mel_step_size:])
    break
  mel_chunks.append(mel[:, start_idx : start_idx + mel_step_size])
  i += 1

print("Length of mel chunks: {}".format(len(mel_chunks)))

full_frames = full_frames[:len(mel_chunks)]

batch_size =wav2lip_batch_size
gen = datagen(full_frames.copy(), mel_chunks)

for i, (img_batch, mel_batch, frames, coords) in enumerate(tqdm(gen, 
                    total=int(np.ceil(float(len(mel_chunks))/batch_size)))):
  if i == 0:
    model = load_model(checkpoint_path)
    print ("Model loaded")

    frame_h, frame_w = full_frames[0].shape[:-1]
    out = cv2.VideoWriter('temp/result.avi', 
                cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_w, frame_h))

  img_batch = torch.FloatTensor(np.transpose(img_batch, (0, 3, 1, 2))).to(device)
  mel_batch = torch.FloatTensor(np.transpose(mel_batch, (0, 3, 1, 2))).to(device)

  with torch.no_grad():
    pred = model(mel_batch, img_batch)

  pred = pred.cpu().numpy().transpose(0, 2, 3, 1) * 255.
  
  for p, f, c in zip(pred, frames, coords):
    y1, y2, x1, x2 = c
    p = cv2.resize(p.astype(np.uint8), (x2 - x1, y2 - y1))

    f[y1:y2, x1:x2] = p
    out.write(f)

out.release()

command = 'ffmpeg -y -i {} -i {} -strict -2 -q:v 1 {}'.format(audiof, 'temp/result.avi', outfile)
subprocess.call(command, shell=platform.system() != 'Windows')



Using cuda for inference.
Reading video frames...
Number of frames available for inference: 862
Extracting raw audio...


  0%|          | 0/10 [00:00<?, ?it/s]

(80, 9402)
Length of mel chunks: 1175



100%|██████████| 54/54 [00:46<00:00,  1.16it/s]


Load checkpoint from: checkpoints/wav2lip_gan.pth
Model loaded


100%|██████████| 10/10 [01:07<00:00,  6.74s/it]


0

### Displaying the result

In [23]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.